[View in Colaboratory](https://colab.research.google.com/github/farhanhubble/sapient-webinars/blob/master/Unsupervised_Learning.ipynb)

## Unsupervised Learning using Matrix Decomposition

Matrix decomposition is a family of powerful unsupervised learning methods. Some examples are principal component analysis (PCA), singlular value decomposition (SVD), independent component analysis (ICA), independent vector ananlysis (IVA), non-negative matrix factorization (NMF) 

### Principal Component Ananlysis 

**Motivation:**

- If every instance in a dataset has $n$ features or equivalently, it is a $n$-dimensional vector, can each instance be approximately represented as a $m$-dimensional vector ($m<<n$)? 


**Question:**

- If such a representation is possible what is a good choice for the $m$ new dimensions?

**Clues:** 
In most cases dimensions are not totally independent so more than one dimensions can be rolled into one. 


**Examples:** 
1. Consider a dataset containing 10s audio clips (e.g. Youtube8M). The features here are made up of  the amplitude of the audio recorded (say) 44100/s. So every clip will be a vector of 44100 x 10 = 441000 or the dataset  could be said to be 441000-dimensional. 

Because of the continuous nature of audio, nearby samples will be similar in magnitude.


2. Consider 1000px x 1000px images. When flattened, every image hass 1,000,000 features but because of the 2D nature of images every pixel will have an intensity that is very likely to be similar to its neighboring pixels. When the image is flattened, nearby pixels get distributed to far away indices but the similarity remains.


In both cases, there is substantial amount of redundancy and the dimension is very large. If we just want to reduce the dimension (at the expense of some information loss), we could subsample the data. For example audio can be resampled to 16000 samples/s or image can be resampled to 400px * 400px. The subsampled data will still have the redundancy mentioned earlier.



### Redundancy can be caused by multiple sources:

- If we know the extent of redundancy and the features that are redundant, we can perhaps combine them using some simple strategy, for example, replacing many redundant features with their mean. However taking means is a local operation, applied to every instance in the dataset, it does not benefit from the information available about the redundancy across all instances.

- Very often we do not know which features are redundant, for example [this tutorial](https://www.cs.princeton.edu/picasso/mats/PCA-Tutorial-Intuition_jp.pdf) gives the example of an object that actually is constrained to move in a 1d line, but whose position is being captured by 3 cameras, each of which records the position of the object on some 2d space. This produces 6 measurements or features for otherwise what would have been described with just one feature. 

![1D spring mass system](https://snag.gy/hmrcH5.jpg)


**By looking at all values of a feature in a dataset, we can better estimate the redundancy in the dataset. Every feature can be treated as a distribution.**

### Measuring Redundancy
- Measuring redundancy requires a definition of redundancy 

 1. If the values in a column are nearly a multiple of values in another column, are the two redundant?
 2. If the values in a column are nearly the squares of values in another column, are they redundant?
 3. If the values in a column are ratios of values in two other columns?
 
 It turns out that redundancy can only be defined based on the problem at hand and our model of the solution.
 
** Example:** Suppose you are trying to build a system that predicts the trajectory of a football. You are estimating the position, velocity, spin rate  and direction of motion of the ball  from a sequence of video frames and then using these as features to predict the trajectory of the ball. Should you model use just the velocity or velocity squared?

It turns out that motion in the presence of air drag depends on the square of the velocity too and hence a model that takes this (domain knowledge) into account and uses what appears to be redundant information will likely outperform a model that uses only the velocity as a feature.


### Lessons
- ** Not only can redundancy be desirable sometimes, it is generally impossible to find redundancy of a general nature. **


Keeping this in mind we set our sight on removing the simplest type of redundancy that is both undesirable as well as easy to detect and that is : ** Covariance / Correlation **

### Covariance
The covariance of two  (feature) vectors $ \vec v_1, \vec v_2 $ is their dot product normalized by their length :  $Cov(v_1,v_2)  = {{{(\vec v_1 - \bar v_1)} \cdot {(\vec v_2 - \bar v_2)}} \over n}$ where $\bar \cdot$ indicates the mean. This is similar to the variance of a single vector, that can be defined as  $Var(v_1) = Cov(v_1,v_1) = {{{(\vec v_1 - \bar v_1)} \cdot {(\vec v_1 - \bar v_1)}} \over n}$. 


### Correlation
The correlation of two vectors is their covariance scaled by the product of their standard deviations defined as $Cor(v_1, v_2) = {Cov(v_1, v_2) \over {\sqrt {Var(v_1)} \sqrt {Var(v_2)} }}$ = $Cor(v_1, v_2) = {Cov(v_1, v_2) \over {\sqrt {Cov(v_1,v_1)} \sqrt {Cov(v_2,v_2)} }}$ 


** It is easy to see that the covariance or correlation is zero if the two vectors are orthogonal and non-zero otherwise. **

The covariance of $n$ feature vectors is a $n \times n$ matrix of the form:

$$
\begin{bmatrix}
Cov(v_1,v_1) && Cov(v_1,v_2) && ... && Cov(v_1,v_n) \\
Cov(v_2,v_1) && Cov(v_2,v_2) && ... && Cov(v_2,v_n) \\
. && . && ... &&. \\
. && . && ... &&. \\
. && . && ... &&. \\
Cov(v_n,v_1) && Cov(v_n,v_2) && ... && Cov(v_n,v_n) \\
\end{bmatrix}
$$

** The idea of Principal Component Analysis is to take a covariance (or correlation) matrix of a dataset and decompose it into  a product of three matrices $U*D*U^T$ ,where the matrix $D$ is nearly diagonal. The columns of matrix U are new dimensions (unit vectors) onto which the old features  can be projected. These unit vectors are called the principal components or eigen-vectors and the diagonal values in D are called the eigenvalues. Since the off-diagonal elements are nearly zero, the eigen vectors have almost zero covariance. **  


In [0]:
import numpy as np

In [0]:
v1 = [1,2,3]
v2 = [4,7,9]
m1 = np.mean(v1)
m2 = np.mean(v2)
print(v1-m1, v2-m2)
np.cov(v1,v2)

In [0]:
v1 = [1,2,3]
v2 = [5,1,-2]
m1 = np.mean(v1)
m2 = np.mean(v2)
print(v1-m1, v2-m2)
np.cov(v1,v2)

In [0]:
v1 = [1,2,3]
v2 = [0.66,0,0.66]
m1 = np.mean(v1)
m2 = np.mean(v2)
print(v1-m1, v2-m2)
np.cov(v1,v2)

### Geometric Interpretation of PCA
Geometrically, PCA gives us new dimensions that maximize variance while minimizing covariance. 

![variance](https://snag.gy/5jQWKe.jpg)

### Applications of PCA

- Decorrelation
- Feature selection 
- Dimensionality Reduction 


### Caveats
- PCA is lossy.
- PCA does not take into account the labels (that's why it is unsupervised) and may make classification harder.

In [0]:
#### The Fashion MNIST dataset

In [0]:
from urllib import request
import matplotlib.pyplot as plt

In [0]:
request.urlretrieve('https://github.com/farhanhubble/sapient-webinars/raw/master/fashion_train.npy','./fashion_train.npy')
request.urlretrieve('https://github.com/farhanhubble/sapient-webinars/raw/master/fashion_test.npy','./fashion_test.npy')

In [0]:
fashion_train = np.load('fashion_train.npy')
fashion_test = np.load('fashion_test.npy')

In [0]:
fashion_train_X , fashion_train_y = fashion_train[:,1:] , fashion_train[:,0]
fashion_test_X , fashion_test_y = fashion_test[:,1:] , fashion_test[:,0]

In [0]:
for i in range(20):
  plt.subplot(4,5,i+1)
  plt.imshow(fashion_train_X[i].reshape([28,28]), cmap='gray')
plt.show()

In [0]:
from sklearn.decomposition import PCA

In [0]:
pca = PCA(0.9)

In [0]:
fashion_train_X_pca = pca.fit_transform(fashion_train_X)
fashion_test_X_pca = pca.transform(fashion_test_X)

In [0]:
## Note that the principal components returned by Sklearn are stored row-wise
## So there are 81 vecctors each of length 784.
print(pca.components_.shape)
print(pca.n_components_, np.cumsum(pca.explained_variance_ratio_))
                                   

#### Eyeballing redundancy in Fashion MNIST


In [0]:
## Measure covariance across columns

## Covariance of original data.
plt.imshow(np.cov(fashion_train_X.T),cmap='gray')
plt.show()

## Covariance of transformed data columns (first 20 only)
plt.imshow(np.cov(fashion_train_X_pca.T[:10]),cmap='gray')
plt.show()

## Covariance of transformed data columns 
plt.imshow(np.cov(fashion_train_X_pca.T),cmap='gray')
plt.show()

#### Plot the first few principal components

In [0]:
for i in range(20):
  plt.subplot(4,5,i+1)
  plt.imshow(pca.components_[i].reshape([28,28]), cmap='gray')
plt.show()

### Applying PCA based dimensionality reduction to classsification

#### Apply PCA before logistic regression.

In [0]:
from sklearn.linear_model import LogisticRegression

In [0]:
lr = LogisticRegression()

In [0]:
## Fitting to raw data takes around 10 minutes and achieves train accuracy of 95.4%
## and test accuracy of 77.6%.
## - High training time
## - Huge overfitting 

# %%time
# lr.fit(fashion_train_X, fashion_train_y)
# print(lr.score(fashion_train_X, fashion_train_y),lr.score(fashion_test_X, fashion_test_y))

In [0]:
## Fitting to PCA'd data is 20X faster and has drastically reduced overfitting.
%%time
lr.fit(fashion_train_X_pca, fashion_train_y)
print(lr.score(fashion_train_X_pca, fashion_train_y),lr.score(fashion_test_X_pca, fashion_test_y))

#### Apply PCA before random forest

In [0]:
from sklearn.ensemble import RandomForestClassifier

In [0]:
rfc = RandomForestClassifier()

In [0]:
%%time
rfc.fit(fashion_train_X, fashion_train_y)
print(rfc.score(fashion_train_X, fashion_train_y),rfc.score(fashion_test_X, fashion_test_y))

In [0]:
%%time
rfc = RandomForestClassifier()
rfc.fit(fashion_train_X_pca, fashion_train_y)
print(rfc.score(fashion_train_X_pca, fashion_train_y),rfc.score(fashion_test_X_pca, fashion_test_y))

### Using PCA based dimensionality reduction for clustering

In [0]:
from sklearn.cluster import KMeans

In [0]:
km = KMeans(n_clusters=10)

In [0]:
cluster_id_fashion_train = km.fit_predict(fashion_train_X)
cluster_id_fashion_pca = km.fit_predict(fashion_train_X_pca)

In [0]:
def get_label_count_by_cluster_id(cluster_id,y): 
  label_count_by_cluster_id = np.zeros([10,10])

  for cid in np.unique(cluster_id):
    for label in np.unique(y):
      label_count_by_cluster_id[cid,label] = np.sum(y[cid == cluster_id] == label)
  
  return label_count_by_cluster_id
    


In [0]:
from scipy.stats import entropy

def get_avg_cluster_impurity(counts_by_cluster_id,cluster_ids):
  impurity = np.zeros([10])
  for cid in np.unique(cluster_ids):
    impurity[cid] = entropy(counts_by_cluster_id[cid,:])

  return (np.mean(impurity))
  

In [0]:
label_count_train = get_label_count_by_cluster_id(cluster_id_fashion_train,fashion_train_y)
label_count_pca = get_label_count_by_cluster_id(cluster_id_fashion_pca,fashion_train_y)

In [0]:
print('Raw')
print(label_count_train)
print(get_avg_cluster_impurity(label_count_train, cluster_id_fashion_train))

print('PCA\'d')
print(label_count_pca)
print(get_avg_cluster_impurity(label_count_pca, cluster_id_fashion_pca))

## Independent Component Analysis

**Motivation:** Use a more powerful measure of redundancy than covariance / correlation.

**The Idea of Statistical Independence:** Suppose we are given two vectors (or distributions) resulting from rolling **biased** dice $n$ times, how can we ascertain if the two vectors are produced by two different (differently biased) dice or the same die? The idea is to use probability theory to test if the joint probabilty distribution of the two distributions and the product of their marginal distributions is almost same. 



#### Testing independence of two distinct dice using joint probability

In [0]:
die1_faces = [1,2,3,4,5,6]
die1_weights = [0.1, 0.2, 0.2, 0.2, 0.2, 0.1]
die2_faces = [1,2,3,4,5,6]
die2_weights = [0.09, 0.17, 0.29, 0.19, 0.2, 0.06]

In [0]:
## Disctinct dice
die1_output = np.random.choice(die1_faces,p=die1_weights,size=1000)
die2_output = np.random.choice(die2_faces,p=die2_weights,size=1000)

In [0]:

plt.hist(die1_output,bins=die1_faces+[7],color='g',histtype='step')
plt.hist(die2_output,bins=die2_faces+[7],color='r',histtype='step')
plt.show()



In [0]:
probs1, _ = np.histogram(die1_output,bins=die1_faces+[7], normed=True)
probs2, _ = np.histogram(die2_output,bins=die2_faces+[7], normed=True)
print(probs1,probs2)

In [0]:
def get_joint_probability(die1, die2):
  joint_probability = np.zeros([6,6])

  for a,b in zip(die1, die2):
    joint_probability[a-1,b-1] += 1
  
  joint_probability /= np.sum(joint_probability)
  
  return joint_probability

jp_distinct = get_joint_probability(die1_output, die2_output)
print(jp_distinct)

In [0]:
def get_prod_probability(dist1, dist2):
  prod_probability = np.zeros([6,6])
  
  for i in range(len(dist1)):
    for j in range(len(dist2)):
      prod_probability[i,j] = dist1[i]*dist2[j]
  
  return prod_probability

pp_distinct = get_prod_probability(probs1,probs2)
print(pp_distinct)

In [0]:
np.mean(np.abs(jp_distinct - pp_distinct))

In [0]:
## Re-roll first die
die1_output_ = np.random.choice(die1_faces,p=die1_weights,size=1000)
probs1_ , _ = np.histogram(die1_output_,bins=die1_faces+[7], normed=True)

jp_non_distinct = get_joint_probability(die1_output, die1_output_)
pp_non_distinct = get_prod_probability(probs1,probs1_)
print(jp_non_distinct)
print(pp_non_distinct)
print(np.mean(np.abs(jp_non_distinct - pp_non_distinct)))

#### Testing independence of two distinct dice using relative entropy aka  [Kullback–Leibler divergence](https://en.wikipedia.org/wiki/Kullback%E2%80%93Leibler_divergence) 

In [0]:
print(entropy(probs1, probs1_))
print(entropy(probs1, probs2))

### ICA finds new dimensions that maximize the statistical independence between the  new dimensions. 

Applications of ICA and its variants

- Image Reflection Removal [1](https://pdfs.semanticscholar.org/c106/d34429cddab7291e02664025fe59cdae846a.pdf) [2](http://webee.technion.ac.il/people/zeevi/papers/38.pdf.pdf)

- Face recognition [1](http://cs229.stanford.edu/proj2007/Rajgarhia-FaceDetectionUsingICA.pdf)

- Medical signal analysis (fMRI, ECG, EEG)

- Blind source audio separation. [1](http://www.kecl.ntt.co.jp/icl/signal/sawada/mypaper/icassp2007Tutorial11.pdf)

### Blind Source Audio Separation with ICA
Let us say we are given three recordings of three persons speaking simultaneously. Each recording is a dimension or feature in the original dataset.
ICA can be used to derive new features (or dimensions) that are more statistically independent with each other than the original dimensions.
The new dimensions will be mostly pure audio from one person with audio from other persons magically removed.

#### Unmix synthetic audio.
We will first mix 3 pure audio recordings in different ratios and generate 3 mixtures. We will then unmix this synthetic mixture using ICA decomposition.

#### Download pure audio from 3 different speakers.
The files are taken from [here](http://www.kecl.ntt.co.jp/icl/signal/sawada/demo/bss2to4/index.html).

In [0]:

request.urlretrieve('https://github.com/farhanhubble/sapient-webinars/raw/master/s1.wav', './s1.wav')
request.urlretrieve('https://github.com/farhanhubble/sapient-webinars/raw/master/s2.wav', './s2.wav')
request.urlretrieve('https://github.com/farhanhubble/sapient-webinars/raw/master/s3.wav', './s3.wav')


#### Create synthetic mixtures.
For synthetic mixing we just add the audios. To create 3 mixtures we try three different (arbitrary) mixing ratios.

In [0]:
from scipy.io import wavfile

In [0]:
### Create 3 different mixtures by adding the pure audio recordings in different ratios.
rate1, s1 = wavfile.read('./s1.wav')
rate2, s2 = wavfile.read('./s2.wav')
rate3, s3 = wavfile.read('./s3.wav')

syn_mix1 = 0.3*s1+0.2*s2+0.5*s3
syn_mix2 = 0.2*s1+0.4*s2+0.4*s3
syn_mix3 = 0.4*s1+0.3*s2+0.3*s3

#### Listen to the pure recordings

In [0]:
from IPython.display import Audio

In [0]:
Audio(s1,rate=rate1)

In [0]:
Audio(s2,rate=rate2)

In [0]:
Audio(s3,rate=rate3)

#### Plot the pure audio recordings and their mixtures.

In [0]:
plt.subplot(1,3,1)
plt.plot(s1,'r')
plt.subplot(1,3,2)
plt.plot(s2,'g')
plt.subplot(1,3,3)
plt.plot(s3,'b')
plt.show()

plt.subplot(1,3,1)
plt.plot(syn_mix1,'c')
plt.subplot(1,3,2)
plt.plot(syn_mix2,'y')
plt.subplot(1,3,3)
plt.plot(syn_mix3,'m')
plt.show()

In [0]:
### Create a single matrix containing the three mixtures in its columns.
syn_audio_mix = np.hstack((syn_mix1.reshape([-1,1]), syn_mix2.reshape([-1,1]), syn_mix3.reshape([-1,1])))

#### Listen to the mixtures.

In [0]:
Audio(syn_mix1,rate=rate1)

In [0]:
Audio(syn_mix2,rate=rate2)

In [0]:
Audio(syn_mix3,rate=rate3)

In [0]:
from sklearn.decomposition import FastICA

In [0]:
ica = FastICA(whiten=True, max_iter=10000, tol=1e-6)

In [0]:
### Unmix the three mixtures to 3 statistically independent components.
syn_audio_unmixed = ica.fit_transform(syn_audio_mix)

#### Listen to unmixed components.
Note that there is no particular order in which the components are generated by ICA. 

In [0]:
Audio(syn_audio_unmixed[:,0], rate=rate3)

In [0]:
Audio(syn_audio_unmixed[:,1], rate=rate3)

In [0]:
Audio(syn_audio_unmixed[:,2], rate=rate3)

#### Unmix real-world audio.
Download 3 audio files, each of which is a mixture of audio (speech) of 3 speakers. The files are avialable [here]( http://www.kecl.ntt.co.jp/icl/signal/sawada/demo/bss2to4/index.html). Unlike the synthetic mixtures, these mixtures were produced by playing three audios in a room and then recording the audio in the room with three different microphones. This mixture is not a simple element-wise sum, rather a convolutive sum as direct audio is mixed with multiple reverberated and delayed audio signals. 

See [this](http://www2.imm.dtu.dk/pubdb/views/edoc_download.php/4924/pdf/imm4924.pdf) paper for different types of audio mixing. 

In [0]:
audio_mix1_url = 'https://github.com/farhanhubble/sapient-webinars/raw/master/x3-1.wav'
audio_mix2_url = 'https://github.com/farhanhubble/sapient-webinars/raw/master/x3-2.wav'
audio_mix3_url = 'https://github.com/farhanhubble/sapient-webinars/raw/master/x3-3.wav'

request.urlretrieve(audio_mix1_url, './mix1.wav')
request.urlretrieve(audio_mix2_url, './mix2.wav')
request.urlretrieve(audio_mix3_url, './mix3.wav')


In [0]:
rate1, mix1 = wavfile.read('./mix1.wav')
rate2, mix2 = wavfile.read('./mix2.wav')
rate3, mix3 = wavfile.read('./mix3.wav')

In [0]:
Audio(mix1, rate=rate1)

In [0]:
Audio(mix2, rate=rate2)

In [0]:
Audio(mix3, rate=rate3)

In [0]:
audio_mixed = np.hstack((mix1.reshape([-1,1]), mix2.reshape([-1,1]), mix3.reshape([-1,1])))

In [0]:
audio_unmixed = ica.fit_transform(audio_mixed)

In [0]:
Audio(audio_unmixed[:,0], rate=rate3)

In [0]:
Audio(audio_unmixed[:,1], rate=rate3)

In [0]:
Audio(audio_unmixed[:,2], rate=rate3)

### ICA caveats

- Unlike PCA, in ICA the number of components has to be guessed.
- For audio decomposition using ICA the number of features/mixtures should be equal to the number of pure sources. 
- For real-world audio decomposition, the mixtures should be first transformed to the frequency domain using short-time frequency transform(STFT).

### How is ICA a decomposition?
The mixing process is 
$X = M \cdot A$

ICA takes a matrix of mixtures $X$ and finds an unmixing matrix $U$ such that $U*X \simeq A$, essentially decomposing a mixture into a mixing matrix and unmixed signals.

## Non-negative Matrix Factorization (NMF)
PCA tries to minimize linear dependence or correlation between features/dimensions while
ICA tries to maximize the, much more stringent, statistical independence between features. NMA on the other hand uses a very relaxed condition to decompose a matrix $V_{m \times n}$ into a product of two matrices $W_{m \times k} \cdot H_{k \times n}$, such that every element in both $W$ and $H$ is non-negative and the intermediate dimenskion $k << n$ (sparsity).

NMF is used in [clustering](https://pdfs.semanticscholar.org/cbb3/1694c3ca105c618c06374475ceb3aa80f9db.pdf) and [collaborative filtering](http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.108.415&rep=rep1&type=pdf).

In [0]:
from sklearn.decomposition import NMF

In [0]:
nmf = NMF()

In [0]:
W = nmf.fit_transform(fashion_test_X)